In [1]:
from pathlib import Path
import os

import torch
from torch import nn
import timm
from skimage import io, transform

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from matplotlib import pyplot as plt

import pandas as pd

from sklearn import metrics

from tqdm.auto import trange, tqdm

import warnings
warnings.filterwarnings("ignore")

dev = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
DATA_ROOT   = Path('./data')
WORKING_DIR = Path('./')
STUDY_PATH = WORKING_DIR / 'studies'

In [3]:
X = pd.read_csv(DATA_ROOT / 'X_train.csv')
y = pd.read_csv(DATA_ROOT / 'train_labels.csv')

In [4]:
def clean_tile(t):
    t = t.replace('\'', '_')
    return ''.join(e for e in t if e.isalnum() or e in '- _¢·v)(')

In [5]:
y['title'] = [clean_tile(x) for x in X['movie_title']]

In [6]:
[x for x in y['title'] if 'Before' in x]

['Before Sunrise',
 'The Life Before Her Eyes',
 'Before Sunset',
 'Me Before You',
 'Before I Go to Sleep',
 'Before Midnight']

In [7]:
class PostersDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, genres_csv, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.titles = [f.split('.')[0] for f in os.listdir(root_dir)]        
        self.genres = genres_csv

    def __len__(self):
        return len(self.genres)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                f'{self.titles[idx]}.jpg')
        img = io.imread(img_name)

        if self.transform:
            img = self.transform(img)

        y = self.genres[self.genres['title'] == self.titles[idx]].drop(['imdb_score', 'title'], axis=1).to_numpy()
        if len(y) == 0:
            print("============")
            print(self.titles[idx])
            print("============")
        return img, y

In [8]:
trasf = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize((224, 224)),
                            ])

train_ds = PostersDataset(y, DATA_ROOT / 'train_posters', transform=trasf)
train_dtld = DataLoader(train_ds, batch_size=256)

In [9]:
a = next(iter(train_dtld))

In [10]:
def imshow(img):
    plt.imshow(torch.movedim(img, 0, 2))

In [11]:
class ImgClass(nn.Module):
    def __init__(self):
        super(ImgClass, self).__init__()
        # self.input = nn.Conv2d(3, 3, 5, 1, 2)
        self.resnet = timm.create_model('resnet34', pretrained=True)
        self.output = nn.Linear(1000, 1)

    def forward(self, x):
        # x = self.input(x)
        x = self.resnet(x)
        return self.output(x).sigmoid()

In [12]:
models = [ImgClass().to(dev) for _ in range(19)]

In [ ]:
N_EPOCHS = 10
loss_fn = nn.BCELoss()
losses = [[] for _ in range(19)]
accs = [[] for _ in range(19)]
optims = [torch.optim.Adam(models[i].parameters(), lr=1e-3) for i in range(19)]

for epoch in trange(N_EPOCHS):
    for X, y in tqdm(train_dtld, leave=False):
        for i, model in enumerate(models):
            X, y = X.to(dev), y.to(dev)
            y_pred = model(X)

            loss = loss_fn(y_pred, y[:, :, i].float())
            
            acc = metrics.accuracy_score(y[:, :, i].cpu().detach().numpy(), y_pred.cpu().detach().numpy() > 0.5)

            model.zero_grad()
            loss.backward()
            optims[i].step()

            losses[i].append(loss.item())
            accs[i].append(acc)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
_ = plt.plot(losses)

In [ ]:
_ = plt.plot(accs)